In [203]:
import pandas as pd
import os
import sklearn
from sklearn.model_selection import train_test_split

In [2]:
data_directory = "F:/Data Science/EDA_with_Sagemaker/dataset"

In [3]:
def import_data(name):
    file = f'{name}'
    data = os.path.join(data_directory, file)
    return pd.read_csv(data)

In [4]:
flights = import_data('flight_price.csv')
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


## Preliminary Analysis

### Checking the dataset for:
- Data-types
- Null values

In [5]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- Total number of columns are 11.
- Missing values in Columns Route and Total_Stops
- Data-type of the columns is not correct

### Check for Duplicates

In [6]:
flights.duplicated().sum()

np.int64(220)

- Data has 220 duplicate values

In [7]:
(   
flights
    .loc[flights.duplicated()]
    .sort_values(['Airline', 'Source', 'Destination', 'Date_of_Journey'])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
9942,Air India,12/03/2019,Banglore,New Delhi,BLR → BOM → NAG → DEL,08:50,10:15 13 Mar,25h 25m,2 stops,No info,12778
1982,Air India,1/06/2019,Delhi,Cochin,DEL → AMD → BOM → COK,19:45,19:15 02 Jun,23h 30m,2 stops,No info,10651
4571,Air India,1/06/2019,Delhi,Cochin,DEL → HYD → BOM → COK,17:15,19:15 02 Jun,26h,2 stops,No info,12970
...,...,...,...,...,...,...,...,...,...,...,...
9216,Jet Airways,9/06/2019,Delhi,Cochin,DEL → IDR → BOM → COK,05:30,04:25 10 Jun,22h 55m,2 stops,No info,13292
7502,SpiceJet,03/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No info,6860
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873


## Observation:
- There are total `220` duplicates which should be removed
- `Total_Stops` and `Duration` are integer columns so changed their datatype to correct format.
- Change the datatype type of `Date_of_Journey`, `Dep_Time`, `Arrival_Time` to DateTime

# Detailed Analysis

In [8]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- Jet Airways Business and Jet Airways are same airlines
- Multiple carriers Premium economy and Multiple carriers are same airlines
- Vistara and Vistara Premium economy are same airlines
    - Make changes and convert them as one airline

In [9]:
# (
# flights
#     .Airline
#     .str.replace('Premium economy', '')
#     .str.replace('Business', '')
#     .unique()
# )

(
flights
    .Airline
    .str.replace('Premium economy', '')
    .str.replace('Business', '')
    .str.title() # Converting into title case
)

0             Indigo
1          Air India
2        Jet Airways
3             Indigo
4             Indigo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

## 2nd Column
- Date_of_Journey

In [10]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

#### Converting dates into correct format for further analysis

In [11]:
pd.to_datetime(flights.Date_of_Journey, dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### 3rd Column 
- Source

In [12]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### 4th Column
- Destination

In [13]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

- No issue with columns Source and Destination

### 5th Column
- Dep_Time

In [14]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

- Checking if time contains values other than numbers and :

In [15]:
(
flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains('[^0-9:]')]
)

Series([], Name: Dep_Time, dtype: object)

In [16]:
pd.to_datetime(flights.Dep_Time, format='%H:%M').dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### 6th Column
- Arrival Time

In [17]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [168]:
(
flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains('[^0-9:]')]
    .str.split(' ', n=1)  # as we also have dates so we are only taking the time from the column and splitting only on first space
    .str[0]
    # .unique()
)

np.int64(0)

### 7th Column
- Duration

In [20]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [33]:
(
flights
    .Duration
    .loc[lambda ser: ~ser.str.contains('m')]
    .unique() # To check if there are unaccurate values or any validation issues in the values
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [36]:
(
flights
    .Duration
    .loc[lambda ser: ~ser.str.contains('h')]
)

6474    5m
Name: Duration, dtype: object

- We can see that there is a flight which is only 5m long so logically it's not possible so we need to drop it from the dataset

In [44]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


In [72]:
(
flights
    .Duration
    .drop(6474)
    .str.split(' ', expand=True) # Splitting and converting it into dataframe using expand
    .set_axis(['hour', 'minutes'], axis=1) # Giving column names
    .assign(
       hour = lambda df: df.hour
        .str.replace('h', '')
        .astype('int')
        .mul(60)
    )
 # using assign function(which is used to alter existing or create new columns) to replace h and m to keep the whole duration in minutes
    .assign(
        minutes = lambda df: df.minutes
        .str.replace('m','')
        .fillna(0)
        .astype('int')
    )
    
    .sum(axis=1)  # converting the columns into one by adding them 
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [88]:
(
flights
    .Duration
    .drop(6474)
    .str.split(' ', expand=True) # Splitting and converting it into dataframe using expand
    .set_axis(['hour', 'minutes'], axis=1) # Giving column names
    .assign(
       hour = lambda df: df.hour
        .str.replace('h', '')
        .astype('int')
        .mul(60)
    )
 # using assign function(which is used to alter existing or create new columns) to replace h and m to keep the whole duration in minutes
    .assign(
        minutes = lambda df: df.minutes
        .str.replace('m','')
        .fillna(0)
        .astype('int')
    )
    
    .sum(axis=1)  # converting the columns into one by adding them 
    .rename('duration_minutes') # naming the column as original
    .to_frame() # converting into frame
    .join(flights.Duration.drop(6474)) # joining to check the values 

)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


### 8th Column
- Total Stops

In [89]:
(
flights
    .Total_Stops
)

0        non-stop
1         2 stops
2         2 stops
3          1 stop
4          1 stop
           ...   
10678    non-stop
10679    non-stop
10680    non-stop
10681    non-stop
10682     2 stops
Name: Total_Stops, Length: 10683, dtype: object

In [90]:
(
flights
    .Total_Stops
    .unique()
    
)

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [109]:
(
flights
    .Total_Stops
    .fillna('0')
    .str.replace('non-stop', '0')
    .str.replace(' stops?', '', regex=True)
    .astype(int)
)

0        0
1        2
2        2
3        1
4        1
        ..
10678    0
10679    0
10680    0
10681    0
10682    2
Name: Total_Stops, Length: 10683, dtype: int64

### 9th Column
- Additional Info

In [111]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

- `No info` and `No Info` are same values will convert them into one

In [113]:
flights.Price

0         3897
1         7662
2        13882
3         6218
4        13302
         ...  
10678     4107
10679     4145
10680     7229
10681    12648
10682    11753
Name: Price, Length: 10683, dtype: int64

- As we have performed the operation and know what changes to make now it's time to clean the dataset

# Data Cleaning 

In [186]:
def convert_into_mint(ser):
    return (
        ser
        .str.split(' ', expand=True) # Splitting and converting it into dataframe using expand
        .set_axis(['hour', 'minutes'], axis=1) # Giving column names
        .assign(
            hour = lambda df: 
            (df
            .hour
            .str.replace('h', '')
            .astype('int')
            .mul(60)
        ),
        # using assign function(which is used to alter existing or create new columns) to replace h and m to keep the whole duration in minutes
            minutes = lambda df: 
            (df
            .minutes
            .str.replace('m','')
            .fillna(0)
            .astype('int')
        )

        )
            .sum(axis=1)  # converting the columns into one by adding them 
        )

In [196]:
def Clean_data(df):
    return (
        df
        .drop(6474)
        .drop_duplicates()
        .rename(columns=str.lower) # Converting all the columns into lower order
        .assign(
            airline = lambda df: 
            (
            df
            .airline
            .str.replace('Premium economy', '')
            .str.replace('Business', '')
            .str.title()
        ),

            date_of_journey = lambda df: pd.to_datetime(df.date_of_journey, dayfirst=True),
            dep_time = lambda df: pd.to_datetime(df.dep_time, format='%H:%M').dt.time,
            arrival_time = lambda df: pd.to_datetime(df.arrival_time).dt.time,
            duration = lambda df: df.duration.pipe(convert_into_mint),
            total_stops = lambda df: 
            df
            .total_stops
            .fillna('0')
            .str.replace('non-stop', '0')
            .str.replace(' stops?', '', regex=True)
            .astype(int),

            additional_info = lambda df: df.additional_info.replace('No info', 'No Info')

        )
        .drop('route',axis=1)
            
    )

In [197]:
flights_data = Clean_data(flights)
flights_data

C:\Users\DELL\AppData\Local\Temp\ipykernel_14276\2892734599.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time = lambda df: pd.to_datetime(df.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0,No Info,12648


In [199]:
flights_data

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0,No Info,12648


In [205]:
X = flights_data.drop('price', axis=1)
y = flights_data.price.copy()

In [207]:
X.shape

(10462, 9)

In [208]:
y.shape

(10462,)

In [209]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [213]:
y_train.shape

(8369,)

In [212]:
y_test.shape

(2093,)

In [214]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [218]:
Y_train.shape

(6695,)

In [217]:
Y_val.shape

(1674,)

In [230]:
def export_data(x,y,name):
    file_name = f'{name}.csv'
    file = os.path.join(data_directory, file_name)

    x.join(y).to_csv(file, index=False)
    return pd.read_csv(file).head()

In [233]:
export_data(X,y,'cleaned_data_original')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1,No Info,13302


In [234]:
export_data(X_train,Y_train,'train')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-21,Banglore,New Delhi,08:55:00,19:10:00,615,1,In-flight meal not included,7832
1,Jet Airways,2019-03-27,Delhi,Cochin,17:30:00,04:25:00,655,1,In-flight meal not included,6540
2,Goair,2019-03-09,Banglore,New Delhi,11:40:00,14:35:00,175,0,No Info,7305
3,Air India,2019-06-12,Kolkata,Banglore,09:25:00,18:30:00,545,1,No Info,8366
4,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,07:40:00,525,1,In-flight meal not included,11087


In [235]:
export_data(X_val,Y_val,'val')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-24,Delhi,Cochin,20:25:00,01:30:00,305,1,No Info,5054
1,Multiple Carriers,2019-06-12,Delhi,Cochin,09:45:00,22:30:00,765,1,No Info,9646
2,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,15:15:00,980,1,In-flight meal not included,11087
3,Multiple Carriers,2019-06-06,Delhi,Cochin,13:00:00,21:00:00,480,1,No Info,13587
4,Jet Airways,2019-05-18,Delhi,Cochin,23:05:00,04:25:00,1760,2,No Info,16704


In [236]:
export_data(x_test,y_test,'test')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Banglore,New Delhi,08:00:00,08:15:00,1455,1,No Info,17996
1,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0,No Info,3873
2,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0,No Info,4462
3,Indigo,2019-06-27,Chennai,Kolkata,19:35:00,21:55:00,140,0,No Info,3597
4,Indigo,2019-05-06,Kolkata,Banglore,15:15:00,17:45:00,150,0,No Info,4804
